In [1]:
!pip -q install inltk

In [2]:
import pandas as pd
import numpy as np
import re
import os
import sentencepiece as spm
import pickle
import pathlib

data_paths = {}
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        data_paths[filename] = os.path.join(dirname, filename)
        print(os.path.join(dirname, filename))

/kaggle/input/telugu-wikipedia-dataset/__results__.html
/kaggle/input/telugu-wikipedia-dataset/custom.css
/kaggle/input/telugu-wikipedia-dataset/__notebook__.ipynb
/kaggle/input/telugu-wikipedia-dataset/telugu_wikipedia_dataset.parquet
/kaggle/input/telugu-wikipedia-dataset/telugu_wiki_links.csv
/kaggle/input/telugu-wikipedia-dataset/__output__.json


In [3]:
working_path = '/kaggle/working'

In [4]:
tel_df = pd.read_parquet(data_paths['telugu_wikipedia_dataset.parquet'])
tel_df.shape, tel_df.columns

((99052, 2), Index(['title', 'text'], dtype='object'))

In [5]:
texts = (tel_df['title'] + ' ' + tel_df['text']).tolist()
texts = [t for t in tel_df['text'] if len(t.strip())>1]
print(len(texts))

94435


In [6]:
def preprocess(article):
    article = re.sub(r'^https?:\/\/.*[\r\n]*', '', article)
    article = article.replace(u'\ufeff',' ')
    article = article.replace(u'\xa0', u' ')
    article = article.replace('  ', ' ');
    article = article.replace(' , ', ', ');
    return article

In [7]:
sentences = []
for t in texts:
    t = t.strip('\\n')
    ts = t.split('\n')
    ts = [s.strip() for s in ts if len(s) > 1]
    ts = [preprocess(s) for s in ts]
    sentences.extend(ts)
print(len(sentences))

1449741


In [8]:
tel_path = os.path.join(working_path, 'TeluguWikiSentences.txt')
with open(tel_path, 'w') as f:
    f.write('\n'.join(sentences))

In [9]:
spm.SentencePieceTrainer.Train(f'--input={tel_path} --model_prefix=telugu_tok --vocab_size=25000')

True

In [10]:
sp = spm.SentencePieceProcessor()
sp.Load("telugu_tok.model")

True

In [11]:
for i in range(0,5):
    tokens = sp.EncodeAsPieces(sentences[i])
    print('\n')
    print(sentences[i])
    print(tokens)



వేములకొండ, తూర్పు గోదావరి జిల్లా, రంపచోడవరం మండలానికి చెందిన గ్రామము.[1]. పిన్ కోడ్: 533 288.
['▁వేముల', 'కొండ', ',', '▁తూర్పు', '▁గోదావరి', '▁జిల్లా', ',', '▁రంపచోడవరం', '▁మండలానికి', '▁చెందిన', '▁గ్రామము', '.[1]', '.', '▁పిన్', '▁కోడ్', ':', '▁533', '▁28', '8.']


ఇది మండల కేంద్రమైన రంపచోడవరం నుండి 12 కి. మీ. దూరం లోను, సమీప పట్టణమైన రాజమహేంద్రవరం నుండి 68 కి. మీ. దూరంలోనూ ఉంది. 2011 భారత జనగణన గణాంకాల ప్రకారం ఈ గ్రామం 75 ఇళ్లతో, 232 జనాభాతో 76 హెక్టార్లలో విస్తరించి ఉంది. గ్రామంలో మగవారి సంఖ్య 117, ఆడవారి సంఖ్య 115. షెడ్యూల్డ్ కులాల సంఖ్య 0 కాగా షెడ్యూల్డ్ తెగల సంఖ్య 219. గ్రామం యొక్క జనగణన లొకేషన్ కోడ్ 587154[2].పిన్ కోడ్: 533288.
['▁ఇది', '▁మండల', '▁కేంద్రమైన', '▁రంపచోడవరం', '▁నుండి', '▁12', '▁కి', '.', '▁మీ', '.', '▁దూరం', '▁లోను', ',', '▁సమీప', '▁పట్టణమైన', '▁రాజమహేంద్రవరం', '▁నుండి', '▁68', '▁కి', '.', '▁మీ', '.', '▁దూరంలోనూ', '▁ఉంది', '.', '▁2011', '▁భారత', '▁జనగణన', '▁గణాంకాల', '▁ప్రకారం', '▁ఈ', '▁గ్రామం', '▁75', '▁ఇళ్లతో', ',', '▁23', '2', '▁జనాభాతో', '▁76', '▁హెక్టార్లలో'

In [12]:
from fastai.text import BaseTokenizer
import sentencepiece as spm

class TeluguTokenizer(BaseTokenizer):
    def __init__(self, lang:str):
        self.lang = lang
        self.sp = spm.SentencePieceProcessor()
        self.sp.Load("telugu_tok.model")
        
    def tokenizer(self, t:str):
        return self.sp.EncodeAsPieces(t)